In [1]:
from os import environ
environ['OMP_NUM_THREADS'] = '12'
import numpy as np
import scipy.linalg as la
from qiskit import QuantumCircuit, Aer, execute
from qiskit.extensions import UnitaryGate
import matplotlib.pyplot as plt

C:\Users\lisis\AppData\Local\Temp\ipykernel_12300\2437713394.py:6: DeprecationWarning: The qiskit.extensions module is deprecated since Qiskit 0.46.0. It will be removed in the Qiskit 1.0 release.
  from qiskit.extensions import UnitaryGate


{\displaystyle \left|\det(U)\right|=1}. 
That is, det ( U ) {\displaystyle \det(U)} will be on the unit circle of the complex plane.

Its eigenspaces are orthogonal.

U can be written as U = eiH, where e indicates the matrix exponential, i is the imaginary unit, and H is a Hermitian matrix.

In [42]:
qubit_num = 2 #число кубитов для вычисления собственного значения

b = np.array([1, 0])
H_input = np.array([[1, -1/3], [-1/3, 1]])
a = np.sqrt(2)
b = np.sqrt(3)
c=1
H_input = np.array([[c, a+1j*b], [a-1j*b, c]])
# U = np.array([[-1+1j, 1+1j], [1+1j, -1+1j]]) / 2
import random
alpha = random.uniform(0, 2*np.pi)
beta = random.uniform(0, 2*np.pi)
delta = random.uniform(0, 2*np.pi)
phi = random.uniform(0, 2*np.pi)

# U = la.expm(2*np.pi*2.62/1.23*1j*H_input)
U = np.array([[np.cos(phi)*np.exp(alpha+beta+delta),
               np.sin(phi)*np.exp(alpha-beta+delta)],
              [-np.sin(phi)*np.exp(alpha+beta-delta),
               np.cos(phi)*np.exp(alpha-beta+delta)]])
# print(U)
# print(U)

eigs, vecs = np.linalg.eig(U)
print(vecs)
# eigsH, vecsH = np.linalg.eig(H_input)
# print(eigsH)
# print(eigs)
# print(vecs)
print(np.vdot(*vecs))

[[-0.99973596 -0.96789138]
 [-0.02297838 -0.25136879]]
0.26626999434626614


In [184]:
def noise_val():
    return np.random.normal(0,0.1,1)[0] #+  1.j*np.random.normal(0,0.3,1)[0]
    # return 0.1#+0.1j

In [185]:
#states
state_zero = np.array([[1.0],[0.0]])
state_one = np.array([[0.0],[1.0]])
#projectors
P0 = np.dot(state_zero, state_zero.T)
P1 = np.dot(state_one, state_one.T)

In [186]:
def multi_kron(*args):
    ret = np.array([[1.0]]).astype('complex')
    for q in args: 
        ret = np.kron(ret, q)
    return ret

def multi_dot(*args):
    ret = np.eye(np.shape(args[0])[0]).astype('complex')
    for q in args:
        ret = np.dot(ret, q)
    return ret

In [187]:
def H():
    return 1/np.sqrt(2) * np.array([[1, 1],
                                    [1, -1]]).astype('complex')
def X():
    return np.array([[0, 1],
                     [1, 0]]).astype('complex')
def Rz(phi):
    return np.array([[np.exp(-1j*phi/2), 0],
                     [0, np.exp(1j*phi/2)]]).astype('complex')
def P(phi):
    return np.array([[1, 0],
                     [0, np.exp(1j*phi)]]).astype('complex')
def Ry(theta):
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2), np.cos(theta/2)]]).astype('complex')
def I():
    return np.array([[1, 0],
                     [0, 1]]).astype('complex')

#for some reson CONTROL and TARGET are changed
def control(control_q, target_q, n_of_qubits, operator):
    list_of_ops_left = []
    list_of_ops_right = []
    stop = False
    for i in range(n_of_qubits):
        if i in control_q:
            list_of_ops_left.append(P0)
            list_of_ops_right.append(P1)
        elif i in target_q:
            if np.shape(U) != (2, 2):
                if not stop:
                    for _ in range(len(target_q)):
                        list_of_ops_left.append(I())
                    list_of_ops_right.append(operator)
                    stop = True
            else:
                list_of_ops_left.append(I())
                list_of_ops_right.append(operator)
        else:
            list_of_ops_left.append(I())
            list_of_ops_right.append(I())
    return multi_kron(*list_of_ops_left) + multi_kron(*list_of_ops_right).astype('complex')

In [358]:
#eigenvalues to phases
def eigenvalues_to_phases(n, u):
    # print(n)
    repetitions = 1#2**(n-1)
    ops = []
    target_qubits_amount = int(np.log2(u.shape[0]))
    for i in range(1, n+1):
        for _ in range(repetitions):
            ops.append(control(control_q=[target_qubits_amount+n-i],
                               target_q=[idx for idx in range(target_qubits_amount)],
                               n_of_qubits=n+target_qubits_amount+1,
                               operator=u**1
                               ))
        repetitions *= 2
    return multi_dot(*ops)

In [359]:

def swap(n_of_qubs,u):
    """n-qubit QFTdagger the first n qubits in circ"""
    # Don't forget the Swaps!
    target_qubits_amount = int(np.log2(u.shape[0]))
    qc = QuantumCircuit(n_of_qubs+target_qubits_amount+1)
    for qubit in range(n_of_qubs//2): #n//2
        qc.swap(qubit+1, n_of_qubs-qubit)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    op = result.get_unitary(qc).data
    return op

def qft(n, u):
    ops = []
    target_qubits_amount = int(np.log2(u.shape[1]))
    hadamards = np.array([I() for _ in range(n+target_qubits_amount+1)])
    for j in range(n): 
        for m in range(j):
            ops.append(
                control(
                control_q=[target_qubits_amount - 1 + n - m],
                target_q=[target_qubits_amount - 1 + n - j],
                n_of_qubits=n + target_qubits_amount+1,
                operator=P(np.pi/float(2**(j-m))),
                )
            )
        hadamards_ = hadamards.copy()
        hadamards_[j+1] = H()
        ops.append(multi_kron(*hadamards_))
    return multi_dot(*ops)

def iqft(n, u):
    ops = []
    target_qubits_amount = int(np.log2(u.shape[1]))
    hadamards = np.array([I() for _ in range(n+target_qubits_amount+1)])
    for j in range(n): 
        for m in range(j):
            ops.append(
                control(
                control_q=[target_qubits_amount - 1 + n - m],
                target_q=[target_qubits_amount - 1 + n - j],
                n_of_qubits=n + target_qubits_amount+1,
                operator=P(-np.pi/float(2**(j-m))),
                )
            )
        hadamards_ = hadamards.copy()
        hadamards_[target_qubits_amount - 1 + n - j] = H()
        ops.append(multi_kron(*hadamards_))
    return multi_dot(*ops)

In [397]:
U = la.expm(1j*H_input)

prime_state = multi_kron(
    np.array([[el] for el in b[:]]),
    *[state_zero for _ in range(qubit_num)],
    state_zero)#.round(decimals=10)


state_prep = multi_kron(
    *[I() for _ in range(int(np.log2(len(b))))],
    *[H() for _ in range(qubit_num)],
    I())#.round(decimals=10)

#calculations
eigs_to_phases = eigenvalues_to_phases(qubit_num, U)#.round(decimals=10)

# print(eigs_to_phases)
# np.savetxt('classic.txt', eigs_to_phases.view(float))       
# real_mtrx = np.loadtxt("classic.txt").view(complex)  
# quantum_mtrx = np.loadtxt("outfile.txt").view(complex)
# print((real_mtrx==quantum_mtrx).all())

iqft_matrix = iqft(qubit_num, U)#.round(decimals=10)
ift = multi_dot(swap(qubit_num, U), iqft_matrix)#.round(decimals=10)

# np.savetxt('classic.txt', ift.view(float))       
# real_mtrx = np.loadtxt("classic.txt").view(complex)  
# quantum_mtrx = np.loadtxt("outfile.txt").view(complex)
# print((real_mtrx==quantum_mtrx).all())
anc_rotation = control(
                control_q=[2],
                target_q=[3],
                n_of_qubits = int(np.log2(U.shape[1])) + qubit_num + 1,
                operator=Ry(np.pi),
                ).round(decimals=10)
# np.savetxt('classic.txt', anc_rotation.view(float))       
# real_mtrx = np.loadtxt("classic.txt").view(complex)  
# quantum_mtrx = np.loadtxt("outfile.txt").view(complex)
# print((real_mtrx==quantum_mtrx).all())

anc2_rotation = control(
                control_q=[1],
                target_q=[3],
                n_of_qubits = int(np.log2(U.shape[1])) + qubit_num + 1,
                operator=Ry(np.pi/3),
                )#.round(decimals=10)
# np.savetxt('classic.txt', anc2_rotation.view(float))       
# real_mtrx = np.loadtxt("classic.txt").view(complex)  
# quantum_mtrx = np.loadtxt("outfile.txt").view(complex)
# print((real_mtrx==quantum_mtrx).all())

qft_matrix = qft(qubit_num, U)#.round(decimals=10)
_qft = multi_dot(qft_matrix, swap(qubit_num, U))#.round(decimals=10)
# np.savetxt('classic.txt', _qft.view(float))       
# real_mtrx = np.loadtxt("classic.txt").view(complex)  
# quantum_mtrx = np.loadtxt("outfile.txt").view(complex)
# print((real_mtrx==quantum_mtrx).all())

U = la.expm(-1j*H_input)
phase_to_zero = eigenvalues_to_phases(qubit_num, U)#.round(decimals=10)
# np.savetxt('classic.txt', phase_to_zero.view(float))       
# real_mtrx = np.loadtxt("classic.txt").view(complex)  
# quantum_mtrx = np.loadtxt("outfile.txt").view(complex)
# print((real_mtrx==quantum_mtrx).all())

state_uncompute = multi_kron(
    *[I() for _ in range(int(np.log2(len(b))))],
    *[H() for _ in range(qubit_num)],
    I())#.round(decimals=10)

final_u = multi_dot(
    state_uncompute,
    phase_to_zero,
    _qft,
    anc2_rotation,
    anc_rotation,
    ift,
    eigs_to_phases,
    state_prep
                    ).round(decimals=5)

np.savetxt('classic.txt', final_u.view(float))       
real_mtrx = np.loadtxt("classic.txt").view(complex)  
quantum_mtrx = np.loadtxt("outfile.txt").view(complex)
print((real_mtrx==quantum_mtrx).all())

# final_state = np.dot(final_u, prime_state)
# rho = np.dot(final_state, np.conj(final_state).T)
# vecs_u

True


In [265]:
max_num = 2**qubit_num
max_prob = 0
eigenvalue = -9999
probs = []
potential_eigs = []
max_bin = 0
for i in range(max_num):
    ops = [I() for _ in range(int(np.log2(len(vecs_u[1]))))]
    bin_int = "{0:b}".format(i).zfill(qubit_num)
    ops.extend([P0 if bit == '0' else P1 for bit in bin_int])
    proj = multi_kron(*ops)
    prob = np.sqrt(np.trace(np.dot(proj, rho))**2)
    probs.append(np.round(prob,3))
    # potential_eigs.append(la.expm(1j*2*np.pi*i/(2**qubit_num)))
    potential_eigs.append(i/(2**qubit_num))
    if prob > max_prob:
        max_prob = prob
        eigenvalue = i/(2**qubit_num)
        max_bin = i
probs = [prob.real for prob in probs]
print(f'calculated phase is {np.round(eigenvalue, 3)}')
print(f'true phases are {np.round([np.arctan(eig.imag/eig.real)/(2*np.pi) for eig in eigs_u], 3)}')
print(f'all probabilitis {probs}')
print(f'for the potential phases {potential_eigs}')
# print(f'dispersion', np.std(probs))

calculated phase is 0.25
true phases are [ 0.25 -0.25]
all probabilitis [0.0, 1.0, 0.0, 0.0]
for the potential phases [0.0, 0.25, 0.5, 0.75]
